In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: gast, tensorboard, absl-py, google-pasta, astor, keras-preprocessing, wrapt, protobuf, six, tensorflow-estimator, keras-applications, numpy, termcolor, wheel, scipy, grpcio, opt-einsum
Required-by: 


In [ ]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random
import time

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

In [ ]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')
# Используя объект model строим эмбеддинги для обучающей выборки

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def read_dataset2(path, sample_size, spliting):
    X = []
    y = []   
    

    image_paths_list = glob(os.path.join(path, 'train', '*.jpg'))
    
    for i in tqdm(range(spliting)):
    
        image_paths_sample = random.sample(image_paths_list, sample_size) #список длинной N из последовательности image_paths_list     
        image_paths_list = list(set(image_paths_list) - set(image_paths_sample))
        
        
        for image_path in image_paths_sample:
            image_name = os.path.basename(image_path)
            image_name_parts = image_name.split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None

            if label:
                y.append(int(label == 'cat'))

            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            resnet_pred_features = model.predict(x)
            resnet_pred_features_np = np.array(resnet_pred_features)

            X.append(resnet_pred_features.flatten())
               
    return np.array(X), y

In [ ]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
features, target = read_dataset2("C:\\Users\\roman\\notebook", 1000, 25)
# Проверьте размерности загруженных данных
print(len(features), len(target))



25000 25000


In [ ]:
# с помощью функции train_test_split делим выборку на train и test в отношении 70/30
# В качестве признаков используем эмбеддинги, полученные на предыдущем шаге
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, train_size = 0.7)

print(np.sum(y_train) / len(y_train), np.sum(y_test) / len(y_test))
len(x_train), len(y_train), len(x_test), len(y_test)

0.4990857142857143 0.5021333333333333


(17500, 17500, 7500, 7500)

Создаем объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [ ]:
# Получаем предсказания на тестовом и на обучающем множестве
train_pred = clf.predict(x_train)
test_pred = clf.predict(x_test)

In [ ]:
# С помощью функции accuracy_score оцениваем результаты
train_score = accuracy_score(y_train, train_pred)
test_score = accuracy_score(y_test, test_pred)
print(train_score, test_score)

1.0 0.9868


Делаем тюнинг гиперпараметров для XGBoost:
1. Подбераем максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подбираем подходящий learning rate
3. С помощью библиотеки hyperopt подбераем остальные гиперпараметры при фиксированных num_iterations и learning_rate.

In [ ]:
parameters = {
    "n_estimators": 65,  
    "learning_rate": 0.35,
}


clf = XGBClassifier(**parameters)
clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.35, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=65, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf, features, target, cv=5))

[0.9858 0.9856 0.9862 0.9888 0.9846]


In [ ]:
#from sklearn.metrics import log_loss
from sklearn import preprocessing

from hyperopt import hp
from hyperopt import fmin, tpe

import xgboost as xgb

X = features
y = target

def xgb_score(params):
    clf = XGBClassifier(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    current_score = cross_val_score(clf, X, y, cv=3).mean()
    #print(current_score, params)
    return -current_score
 

space_params = {
            #constant params   
            'n_estimators': 65,  
            'learning_rate': 0.35,

    
            'booster': 'gbtree',             
            'objective': 'binary:logistic',
            'tree_method': 'auto',
            'verbosity': None, 
            'subsample': 1,
    
            #changable                     
            'colsample_bytree': hp.quniform('colsample_bytree', 0.05, 1, 0.05),
            'colsample_bylevel': hp.quniform('colsample_bylevel', 0.05, 1, 0.05),
            'colsample_bynode': hp.quniform('colsample_bynode', 0.05, 1, 0.05),   
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),    
            'max_delta_step':  hp.choice('max_delta_step', np.arange(1, 10, dtype=int)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, dtype=int)), 
}

 
best = fmin(xgb_score, space_params, tpe.suggest, 25)
print('best:', best)

100%|███████████████████████████████████████████████| 25/25 [12:10<00:00, 29.20s/trial, best loss: -0.9864800015566737]
best: {'colsample_bylevel': 0.7000000000000001, 'colsample_bynode': 0.45, 'colsample_bytree': 0.75, 'gamma': 0.9500000000000001, 'max_delta_step': 4, 'max_depth': 12, 'min_child_weight': 1}


In [ ]:
const_params = {
            #constant params   
            'n_estimators': 65,  
            'learning_rate': 0.35,

    
            'booster': 'gbtree',             
            'objective': 'binary:logistic',
            'tree_method': 'auto',
            'verbosity': None, 
            'subsample': 1,
}


clf = XGBClassifier(**best, **const_params)
clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7000000000000001, colsample_bynode=0.45,
              colsample_bytree=0.75, gamma=0.9500000000000001, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.35, max_delta_step=4, max_depth=12,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=65, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='auto',
              validate_parameters=False, verbosity=None)

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf, features, target, cv=5))

[0.9842 0.9854 0.9866 0.987  0.9832]


In [ ]:
with open("C:\\Users\\roman\\notebook\\features_names.txt", "r") as file:
    contents = file.readlines()

for i in range(len(contents)):
    contents[i] = contents[i].strip().replace('\n', '').replace('\r', '')
    
important_feature = list(zip(contents, clf.feature_importances_))

In [ ]:
important_feature

[('tench, Tinca tinca', 0.0),
 ('goldfish, Carassius auratus', 0.00039063854),
 ('great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
  0.0008339753),
 ('tiger shark, Galeocerdo cuvieri', 0.00042843525),
 ('hammerhead, hammerhead shark', 0.0),
 ('electric ray, crampfish, numbfish, torpedo', 0.0),
 ('stingray', 0.00061104476),
 ('cock', 0.0019942166),
 ('hen', 0.00047332628),
 ('ostrich, Struthio camelus', 0.0),
 ('brambling, Fringilla montifringilla', 0.0),
 ('goldfinch, Carduelis carduelis', 0.0),
 ('house finch, linnet, Carpodacus mexicanus', 0.0013960021),
 ('junco, snowbird', 0.0),
 ('indigo bunting, indigo finch, indigo bird, Passerina cyanea', 0.0),
 ('robin, American robin, Turdus migratorius', 0.00061104476),
 ('bulbul', 0.00072527915),
 ('jay', 0.0020775115),
 ('magpie', 0.0004531137),
 ('chickadee', 0.0),
 ('water ouzel, dipper', 0.00061104476),
 ('kite', 0.0),
 ('bald eagle, American eagle, Haliaeetus leucocephalus', 0.0),
 ('vulture', 0.001

In [ ]:
important_feature.sort(reverse=True)

In [ ]:
important_feature.sort(key=lambda i: i[1], reverse=True)
 


In [ ]:
for i in range(20):
    print(important_feature[i])

('Egyptian cat', 0.100030966)
('carton', 0.08039631)
('lynx, catamount', 0.07726841)
('tiger cat', 0.07470517)
('Siamese cat, Siamese', 0.048744358)
('washbasin, handbasin, washbowl, lavabo, wash-hand basin', 0.037516195)
('police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria', 0.0222891)
('Persian cat', 0.019606045)
('radiator', 0.014337252)
('jaguar, panther, Panthera onca, Felis onca', 0.0136743095)
('Chesapeake Bay retriever', 0.009671416)
('jeep, landrover', 0.009242003)
('sidewinder, horned rattlesnake, Crotalus cerastes', 0.007954394)
('tabby, tabby cat', 0.0073042535)
('amphibian, amphibious vehicle', 0.0070373137)
('Angora, Angora rabbit', 0.006894762)
('redbone', 0.006854906)
('coral fungus', 0.0066337488)
('beagle', 0.006591196)
('custard apple', 0.0063715447)
